## Image网 Submission `192x192`

This contains a submission for the Image网 leaderboard in the `128x128` category.

In this notebook we:
1. Train on 1 pretext task: 
  - Train a network to do image inpatining on Image网's `/train`, `/unsup` and `/val` images. 
2. Train on 4 downstream tasks:
  - We load the pretext weights and train for `5` epochs.
  - We load the pretext weights and train for `20` epochs.
  - We load the pretext weights and train for `80` epochs.
  - We load the pretext weights and train for `200` epochs.
  
Our leaderboard submissions are the accuracies we get on each of the downstream tasks.

In [1]:
import json
import torch

import numpy as np

from functools import partial

from fastai2.layers import Mish, MaxPool, LabelSmoothingCrossEntropy
from fastai2.learner import Learner
from fastai2.metrics import accuracy, top_k_accuracy
from fastai2.basics import DataBlock, RandomSplitter, GrandparentSplitter, CategoryBlock

from fastai2.optimizer import ranger, Adam, SGD, RMSProp

from fastai2.vision.all import *
from fastai2.vision.core import *
from fastai2.vision.augment import *
from fastai2.vision.learner import unet_learner, unet_config
from fastai2.vision.models.xresnet import xresnet50, xresnet34

from fastai2.data.transforms import Normalize, parent_label
from fastai2.data.external import download_url, URLs, untar_data


from fastcore.utils import num_cpus

from torch.nn import MSELoss
from torchvision.models import resnet34

## Pretext Task: Image Inpainting

In [2]:
# We create this dummy class in order to create a transform that ONLY operates on images of this type
# We will use it to create all input images
class PILImageInput(PILImage): pass

class RandomCutout(RandTransform):
    "Picks a random scaled crop of an image and resize it to `size`"
    split_idx = None
    def __init__(self, min_n_holes=5, max_n_holes=10, min_length=5, max_length=50, **kwargs):
        super().__init__(**kwargs)
        self.min_n_holes=min_n_holes
        self.max_n_holes=max_n_holes
        self.min_length=min_length
        self.max_length=max_length
        

    def encodes(self, x:PILImageInput):
        """
        Note that we're accepting our dummy PILImageInput class
        fastai2 will only pass images of this type to our encoder. 
        This means that our transform will only be applied to input images and won't
        be run against output images.
        """
        
        n_holes = np.random.randint(self.min_n_holes, self.max_n_holes)
        pixels = np.array(x) # Convert to mutable numpy array. FeelsBadMan
        h,w = pixels.shape[:2]

        for n in range(n_holes):
            h_length = np.random.randint(self.min_length, self.max_length)
            w_length = np.random.randint(self.min_length, self.max_length)
            h_y = np.random.randint(0, h)
            h_x = np.random.randint(0, w)
            y1 = int(np.clip(h_y - h_length / 2, 0, h))
            y2 = int(np.clip(h_y + h_length / 2, 0, h))
            x1 = int(np.clip(h_x - w_length / 2, 0, w))
            x2 = int(np.clip(h_x + w_length / 2, 0, w))
           
            pixels[y1:y2, x1:x2, :] = 0
            
        return Image.fromarray(pixels, mode='RGB')

In [6]:
torch.cuda.set_device(4)

In [7]:
# Default parameters
gpu=None
lr=1e-2
size=128
sqrmom=0.99
mom=0.9
eps=1e-6
epochs=15
bs=64
mixup=0.
opt='ranger',
arch='xresnet50'
sh=0.
sa=0
sym=0
beta=0.
act_fn='Mish'
fp16=0
pool='AvgPool',
dump=0
runs=1
meta=''

In [8]:
# Chosen parameters
lr=8e-3
sqrmom=0.99
mom=0.95
eps=1e-6
bs=64 
opt='ranger'
sa=1
fp16=1               #NOTE: My GPU cannot run fp16 :'(
arch='xresnet50' 
pool='MaxPool'

gpu=0

# NOTE: Normally loaded from their corresponding string
m = xresnet34
act_fn = Mish
pool = MaxPool

In [10]:
def get_dbunch(size, bs, sh=0., workers=None):
    if size<=160: 
        path = URLs.IMAGEWANG_160
    else: 
        path = URLs.IMAGEWANG
    source = untar_data(path)
    if workers is None: workers = min(8, num_cpus())
    #CHANGE: Input is ImageBlock(cls=PILImageInput)
    #CHANGE: Output is ImageBlock
    #CHANGE: Splitter is RandomSplitter (instead of on /val folder)
    
    item_tfms=[RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5), RandomCutout]
#     batch_tfms=RandomErasing(p=0.9, max_count=3, sh=sh) if sh else None
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]
    
    dblock = DataBlock(blocks=(ImageBlock(cls=PILImageInput), ImageBlock),
                       splitter=RandomSplitter(0.1),
                       get_items=get_image_files, 
                       get_y=lambda o: o,
                      item_tfms=item_tfms,
                      batch_tfms=batch_tfms)
    return dblock.dataloaders(source, path=source, bs=bs, num_workers=workers)

In [11]:
name = 'imagewang_inpainting_80_192.pth'

In [12]:
# Use the Ranger optimizer
opt_func = partial(ranger, mom=mom, sqr_mom=sqrmom, eps=eps, beta=beta)

In [13]:
size = 192
bs = 64

dbunch = get_dbunch(size, bs, sh=sh)
#CHANGE: We're predicting pixel values, so we're just going to predict an output for each RGB channel
dbunch.vocab = ['R', 'G', 'B']

In [14]:
len(dbunch.train.dataset), len(dbunch.valid.dataset)

(23714, 2634)

In [15]:
dbunch.show_batch()

In [16]:
learn = unet_learner(dbunch, partial(m, sa=sa), pretrained=False, opt_func=opt_func, metrics=[], loss_func=MSELoss()).to_fp16()
cbs = MixUp(mixup) if mixup else []
learn.fit_flat_cos(80, lr, wd=1e-2, cbs=cbs)




epoch,train_loss,valid_loss,time
0,0.057352,0.049526,02:51
1,0.043885,0.042585,02:42
2,0.040033,0.037075,02:43
3,0.035115,0.044547,02:42
4,0.034318,0.043657,02:42
5,0.032870,0.032451,02:42
6,0.031276,0.033167,02:44
7,0.030827,0.032046,02:42
8,0.030040,0.030323,02:42
9,0.029531,0.029690,02:42


In [17]:
# I'm not using fastai2's .export() because I only want to save 
# the model's parameters. 
torch.save(learn.model[0].state_dict(), name)

## Downstream Task: Image Classification

In [18]:
def get_dbunch(size, bs, sh=0., workers=None):
    if size<=224: 
        path = URLs.IMAGEWANG_160
    else: 
        path = URLs.IMAGEWANG
    source = untar_data(path)
    if workers is None: workers = min(8, num_cpus())
    item_tfms=[RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5)]
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]
#     batch_tfms=RandomErasing(p=0.9, max_count=3, sh=sh) if sh else None
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=GrandparentSplitter(valid_name='val'),
                       get_items=get_image_files, get_y=parent_label,
                      item_tfms=item_tfms, batch_tfms=batch_tfms)
    return dblock.dataloaders(source, path=source, bs=bs, num_workers=workers,
                             )#item_tfms=item_tfms, batch_tfms=batch_tfms)

In [19]:
dbunch = get_dbunch(size, bs, sh=sh)

In [20]:
m_part = partial(m, c_out=20, act_cls=torch.nn.ReLU, sa=sa, sym=sym, pool=pool)

### 5 Epochs

In [21]:
epochs = 5
runs = 5

In [22]:
for run in range(runs):
        print(f'Run: {run}')
                        
        ch = nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(512, 20))
        learn = cnn_learner(dbunch, m_part, opt_func=opt_func, pretrained=False,
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy(),
                config={'custom_head':ch})
        
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
#         # Load weights generated from training on our pretext task
        state_dict = torch.load(name)
        learn.model[0].load_state_dict(state_dict)
        
        learn.unfreeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

Run: 0


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.542418,3.106386,0.072792,0.585900,00:32
1,1.349099,2.616639,0.209723,0.660219,00:33
2,1.240665,2.486085,0.224994,0.785696,00:32
3,1.180982,2.793860,0.156019,0.599389,00:33
4,1.053915,2.109797,0.407992,0.847544,00:33


Run: 1


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.542611,2.911708,0.120387,0.532196,00:32
1,1.353215,3.107859,0.097480,0.620260,00:32
2,1.252238,2.656793,0.229066,0.729193,00:32
3,1.150990,2.814738,0.250445,0.705014,00:32
4,1.036216,2.096101,0.414864,0.843726,00:32


Run: 2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.513661,2.970589,0.090863,0.569102,00:32
1,1.332517,2.662944,0.192415,0.727412,00:32
2,1.258053,2.342783,0.289896,0.806312,00:32
3,1.184687,2.497671,0.256299,0.740137,00:32
4,1.032743,2.080947,0.428608,0.856707,00:32


Run: 3


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.516022,2.699145,0.156019,0.628150,00:32
1,1.345843,3.017068,0.118096,0.563502,00:32
2,1.248484,3.114703,0.118351,0.590736,00:32
3,1.178159,2.321338,0.308475,0.811148,00:33
4,1.035077,2.137833,0.403156,0.853907,00:32


Run: 4


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.526749,4.076327,0.028506,0.295750,00:33
1,1.326773,2.829698,0.217104,0.651820,00:32
2,1.264212,2.312086,0.322983,0.803258,00:33
3,1.186203,2.569664,0.227030,0.757954,00:33
4,1.051646,2.137724,0.398575,0.840417,00:33


* Run 1: 0.362942
* Run 2: 0.372868
* Run 3: 0.342326
* Run 4: 0.360143
* Run 5: 0.357088

Accuracy: **35.91%**

### 20 Epochs

In [23]:
epochs = 20
runs = 3

In [24]:
for run in range(runs):
        print(f'Run: {run}')
                        
        ch = nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(512, 20))
        learn = cnn_learner(dbunch, m_part, opt_func=opt_func, pretrained=False,
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy(),
                config={'custom_head':ch})
        
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
#         # Load weights generated from training on our pretext task
        state_dict = torch.load(name)
        learn.model[0].load_state_dict(state_dict)
        
        learn.unfreeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

Run: 0


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.511677,3.353975,0.055230,0.514126,00:33
1,1.332708,2.771486,0.162382,0.638330,00:32
2,1.250372,2.589838,0.240519,0.765080,00:33
3,1.167782,2.718557,0.213795,0.711631,00:32
4,1.129539,2.027188,0.419700,0.867142,00:32
5,1.084516,2.104485,0.412319,0.852380,00:33
6,1.046261,2.067837,0.432680,0.852380,00:32
7,1.008556,2.126681,0.436243,0.843726,00:32
8,0.996373,1.988299,0.469840,0.878595,00:32
9,0.965702,1.939730,0.500891,0.885976,00:32


Run: 1


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.525088,3.252865,0.074319,0.456350,00:32
1,1.340215,3.102463,0.087045,0.530415,00:33
2,1.241469,2.819765,0.192415,0.702214,00:33
3,1.179706,2.533377,0.243319,0.713922,00:32
4,1.110811,2.238779,0.341817,0.847289,00:32
5,1.073689,2.285269,0.372359,0.826928,00:33
6,1.058432,3.032372,0.189107,0.684144,00:32
7,1.023077,2.019418,0.451769,0.854670,00:32
8,0.999447,1.965320,0.462968,0.873759,00:32
9,0.957765,1.932519,0.497582,0.864342,00:32


Run: 2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.537495,3.621848,0.038687,0.382540,00:32
1,1.344288,2.859647,0.166709,0.649529,00:32
2,1.249339,2.415671,0.298295,0.755663,00:32
3,1.177180,2.276014,0.340290,0.833800,00:33
4,1.151552,2.362281,0.328837,0.769152,00:32
5,1.083822,2.493981,0.312802,0.774243,00:32
6,1.049237,2.272494,0.370578,0.829219,00:32
7,1.018299,1.933839,0.465767,0.902011,00:33
8,1.004576,1.880365,0.507254,0.891321,00:33
9,0.978322,1.974430,0.461950,0.877832,00:32


* Run 1: 0.592263
* Run 2: 0.588445
* Run 3: 0.595571

Accuracy: **59.21%**

## 80 epochs

In [25]:
epochs = 80
runs = 1

In [26]:
for run in range(runs):
        print(f'Run: {run}')
                        
        ch = nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(512, 20))
        learn = cnn_learner(dbunch, m_part, opt_func=opt_func, pretrained=False,
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy(),
                config={'custom_head':ch})
        
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
#         # Load weights generated from training on our pretext task
        state_dict = torch.load(name)
        learn.model[0].load_state_dict(state_dict)
        
        learn.unfreeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

Run: 0


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.527069,3.043183,0.089590,0.512853,00:32
1,1.330183,3.157793,0.101807,0.511581,00:32
2,1.232227,2.571270,0.223467,0.748028,00:32
3,1.168615,2.498776,0.253245,0.753627,00:32
4,1.136587,2.392141,0.288878,0.748028,00:32
5,1.081599,3.120572,0.199542,0.805039,00:32
6,1.053869,2.367441,0.314584,0.792568,00:32
7,1.007538,2.266969,0.368033,0.805803,00:33
8,0.992632,2.096919,0.425554,0.849326,00:32
9,0.988876,1.963978,0.471876,0.864851,00:32


Accuracy: **61.44%**

### 200 epochs

In [27]:
epochs = 200
runs = 1

In [28]:
for run in range(runs):
        print(f'Run: {run}')
                        
        ch = nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(512, 20))
        learn = cnn_learner(dbunch, m_part, opt_func=opt_func, pretrained=False,
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy(),
                config={'custom_head':ch})
        
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
#         # Load weights generated from training on our pretext task
        state_dict = torch.load('imagewang_inpainting_15_epochs_nopretrain.pth')
        learn.model[0].load_state_dict(state_dict)
        
        learn.unfreeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

Run: 0


FileNotFoundError: [Errno 2] No such file or directory: 'imagewang_inpainting_15_epochs_nopretrain.pth'

Accuracy: **59.18%**